# **Volcano Daily Report - Automatic Script**

#### **A script to automatically query whether volcanoes meet the thresholds for "swarm" activity, and produce data products accordingly**

**Scroll to the bottom for outputs**

This is a beta for the script. Feel free to run it but it has not been tested for reliability - For any questions please contact James


In [ ]:
#import relevant modules
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
client = Client("http://service-nrt.geonet.org.nz") #For old events - "GEONET"
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as md
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
from mpl_toolkits.basemap import Basemap

Below defines the date. This automatically pulls "today" UTC, which if run before noon, will be the correct day in UTC

In [ ]:
today = str(UTCDateTime.now())[0:10] + " 17:00:00" # WILL NEED TO BE UPDATED WITH DAYLIGHT SAVINGS
yesterday = str(UTCDateTime.now()-86400)[0:10] + " 17:00:00"

Volcano information section. Edit the equivalent numbers if the definitions of boxes change. Kermadec isles need s a fix I think. Obspy doesn't like negative and ositive longitude

In [ ]:
volcano_list = ["Auckland", "Kermadec Islands", "Mayor Island", "Ngauruhoe",
                "Northland", "Okataina", "Rotorua", "Ruapehu", "Taranaki",
                "Taupo", "Tongariro", "White Island"]
#Lat/long boundaries of said volcanic areas. Must retain current order or will not work - 
# first values correspond to Auckland, second to Kermies etc. 
min_lat = [-37.17, -32.9, -37.53, -39.21, -36.03, -38.34, -38.20, -39.5,
           -39.67, -39.08, -39.21, -38]
max_lat = [-36.57, -24.0, -37.04, -39.08, -34.89, -37.95, -37.98, -39.18,
           -38.93, -38.5, -39.08, -37.34]
min_lon = [174.45, -180, 175.87, 175.56, 173.38, 176.32, 176.12, 175.32,
           173.60, 175.56, 175.56, 176.69]
max_lon = [175.27, -175.8, 176.64, 175.72, 174.75, 176.81, 176.42, 175.77,
           174.49, 176.24, 175.72, 177.40]
stations = ["MKAZ", "GLKZ", "MYRZ", "OTVZ", "OUZ", "TARZ", "UTU", "FWVZ",
            "KHEZ", "RATZ", "TMVZ", "WIZ"] 
# thinking of using for SP plots - currently only using WIZ
volc_df = pd.DataFrame({"volcano_name":volcano_list, "min_lat":min_lat, 
                        "max_lat":max_lat, "min_lon":min_lon, "max_lon":max_lon})

### **Functions for graphs follow**


Much more efficient to define these here than below. I suspect as we get better at writing functions I can make this prettier.

#### Magnitude and depth versus time plots:

In [ ]:
def magtime(catalog, volc, starttime): 
    #combining mag and depth versus time in one graph
    mag, origtime, depth = ([] for i in range(3)) #creates blank lists to iterate over
    for a in range(len(catalog)): 
    #for loop iterating over Obspy Catalog for mag, origin time and depth
        mag.append(catalog[a].preferred_magnitude().mag)
        for b in range(len(catalog[a].origins)): 
            origtime.append(catalog[a].origins[b].time) 
            depth.append(catalog[a].origins[b].depth)
    depth = [x/1000 for x in depth] #conversion to km
    matdates = md.datestr2num([str(d) for d in origtime]) # Conversion into matplotlib dates 
    fig, ax = plt.subplots(2, 1, figsize=(10,10)) #defining figure size and sub plots
    hours = md.HourLocator(interval = 2) 
    h_fmt = md.DateFormatter("%H:%M:%S") #intervals for axes and format
    days = md.HourLocator(interval = 12) 
    d_fmt = md.DateFormatter("%Y-%m-%d")
    def style(a): #function defines styles common to both plots, called by style()
        plt.xticks(rotation = 70)
        fig.autofmt_xdate()
        ax[a].grid()
        if UTCDateTime(today).isoweekday() == 7:
            ax[a].xaxis.set_major_locator(days)
            ax[a].xaxis.set_major_formatter(d_fmt)
        else:
            ax[a].xaxis.set_major_locator(hours)
            ax[a].xaxis.set_major_formatter(h_fmt)            
        ax[a].set_xlim(xmin = md.datestr2num(starttime), xmax = md.datestr2num(today))
    #Magnitude versus time
    ax[0].plot_date(matdates, mag, c = 'r', marker = 'D', ms = 5, mew = 1, mec = "black") 
    style(0)
    ax[0].set_ylim(ymin = 0, ymax = 6)
    ax[0].set_ylabel("Magnitude")
    #Depth versus time
    ax[1].plot_date(matdates, depth, c = "b", marker = "D", ms = 5, mew = 1, mec = "black")
    style(1)
    ax[1].set_ylabel("Depth (km)")
    plt.gca().invert_yaxis()
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    fig.suptitle("Magnitude and depth versus time: " + volc,  fontsize = 16)
    plt.xlabel("Day starting " + str(UTCDateTime.now()- 86400)[0:10] + " UTC") 
    return plt.show()



#### Map of quake locations defined by magnitude:



In [ ]:
def latlon(catalog, volc, tuples): 
    lat, lon, mag = ([] for i in range(3))
    for a in range(len(catalog)): #pulls lats, longs and magnitude
        for b in range(len(catalog[a].origins)): 
            lat.append(catalog[a].origins[b].latitude) 
            lon.append(catalog[a].origins[b].longitude)
            mag.append(catalog[a].preferred_magnitude().mag)
    #function to create values for circle size. Truncated to bin i.e less than 1, 1-2 etc. 
    #Then the exponential and doubling makes #larger earthquakes exponentially larger
    #circles to help them stand out. Values can be changed easily if different proportions wanted.
    magr = [(int(str(x)[0])+1)**2.5 * 2 for x in mag]                                             
    for a in range(len(tuples)): #function to find which volcanic region to use as a basemap
        coords = volc_df.loc[volc_df["volcano_name"] == volc]
    fig, ax = plt.subplots(figsize = (13,13)) 
    map = Basemap(projection = "merc", llcrnrlon = coords["min_lon"], llcrnrlat =  coords["min_lat"],
                  urcrnrlon =  coords["max_lon"], urcrnrlat =  coords["max_lat"], epsg = 2193)
    map.arcgisimage(service = "World_Topo_Map", xpixels = 1000) #closes to Geonet background map
    x, y = map(lon, lat) #converts lat/long lists into mapped values
    scatter = ax.scatter(x, y, s = magr * 200, marker = "o", c = "r", 
                         edgecolors = "black", linewidths = 0.6, alpha = 0.6) 
    #can fiddle to change colors etc.
    plt.title("map")
    return plt.show()


#### SP plot - only currently configured for White Island, and currently configured for WIZ - easy enough to change to WSRZ is that's preferred.


In [ ]:
def sp(cat, starttime):
    data = pd.DataFrame([]) #empty dataframe
    for a in range(len(cat)):
        for b in range(len(cat[a].picks)):
            if cat[a].picks[b].waveform_id.station_code == "WIZ": #or WSRZ
                data = data.append(pd.DataFrame({"time": cat[a].picks[b].time, 
                "phase": cat[a].picks[b].phase_hint,   
                "event": cat[a].resource_id}, index=[0]), ignore_index =True)
    spdict = [] #empty list, to have a dictionary iterated into of P & S values
    for a in range(len(data)):
        try:
            # first test is to see if two rows in the dataframe are the same events 
            # (i.e a p and and s pick).
            # second test to see if the first value listed is a p pick
            if data["event"][a] == data["event"][a+1] and data["phase"][a] == 'P': 
                #S-P calculated as a new dictionary value
                spdict.append({"sp":data["time"][a+1] - data["time"][a] ,"time":data["time"][a]})
                #as above but accounts for when S ends up first in dataframe:
            elif data["event"][a] == data["event"][a+1] and data["phase"][a] == 'S': 
                spdict.append({"sp":data["time"][a] - data["time"][a+1], "time":data["time"][a]})
        except:
            pass   
    df = pd.DataFrame(spdict) #converts list of dictionaries in above for loop into a dataframe
    df = df[df.sp !=0]
    matdates = md.datestr2num([str(d) for d in df["time"]]) # Conversion into matplotlib dates 
    hours = md.HourLocator(interval = 2) 
    h_fmt = md.DateFormatter("%H:%M:%S") #intervals for axes and format
    days = md.HourLocator(interval = 12) 
    d_fmt = md.DateFormatter("%Y-%m-%d")
    fig, ax = plt.subplots(figsize = (10,5))
    ax.plot_date(matdates,  df["sp"], c = "r", marker = "D", ms = 5, mew = 1, mec = "black")
    if UTCDateTime(today).isoweekday() == 7:
        ax.xaxis.set_major_locator(days)
        ax.xaxis.set_major_formatter(d_fmt)
    else:
        ax.xaxis.set_major_locator(hours)
        ax.xaxis.set_major_formatter(h_fmt)
    plt.xticks(rotation = 70)
    ax.grid()
    ax.set_ylabel("S-P Time")
    ax.set_xlim(xmin = md.datestr2num(starttime), xmax = md.datestr2num(today))
    fig.autofmt_xdate()
    ax.set_xlabel("Day starting " + str(UTCDateTime.now()- 86400)[0:10] + " UTC")
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.title("S-P plot: White Island")
    return plt.show()

#### Histogram of magnitudes:

In [ ]:
def histmags(catalog, volc):
    mag = []
    for a in range(len(catalog)):
        mag.append(catalog[a].preferred_magnitude().mag)
    fig, ax = plt.subplots(figsize=(10,5))
    ax.hist(mag, bins  = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6], rwidth = 0.95) 
    #bins can be changed if wanted 
    ax.set_ylabel("Count")
    ax.set_xlabel("Magnitude")
    ax.xaxis.set_minor_locator(MultipleLocator(0.5))
    plt.title("Frequency count by magnitude: " + volc + " " + str(UTCDateTime.now()- 86400)[0:10] + 
              " UTC")
    plt.grid(True)
    return plt.show()

This box runs the rest of the code - running Obspy Catalog pulls to "get_events" for each lat/long box. Returns "No Earthquakes Found" if it fails to find any that fit the criteria. Might remove that in following iterations, or instead make it print out it even if only one is found.

In [ ]:
print("Report for: " + today)
volc_copy, twoplus, weekly = ([] for i in range(3)) 
# Empty list for appending. For loop running a obspy "get events" function to
# pull every even within each of the lat/long boxes, storing them as tuples of
# volcano + events within a larger list
for a in range(len(volcano_list)):
    try:
        volc_copy.append((volcano_list[a], 
        client.get_events(starttime = UTCDateTime(today) - 86400,
        endtime = UTCDateTime(today), minlatitude = min_lat[a],
        maxlatitude = max_lat[a], minlongitude = min_lon[a],
        maxlongitude = max_lon[a], maxdepth = 30)))
    except:
        #if no quakes are found this will print instead
        print(volcano_list[a] + " No earthquakes found") 
print("\n") # new line for good formatting.
for volc in range(len(volc_copy)):
    if len(volc_copy[volc][1]) > 0:
        print(volc_copy[volc][0] +": " + str(len(volc_copy[volc][1])) + 
        " Earthquake(s) in past 24h")
#determining if there are more than 10 events within the specified time period.
anymag = [x for x in volc_copy if len(x[1]) > 10]
#Below is same as the previous function but with the "minmagnitude" parameter
#set to 2, to restrict to only quakes larger than this.
for a in range(len(volcano_list)):
    try:
        twoplus.append((volcano_list[a],
        client.get_events(starttime = UTCDateTime(today) - 86400, 
        endtime = UTCDateTime(today), minlatitude = min_lat[a], 
        maxlatitude = max_lat[a], minlongitude = min_lon[a], 
        maxlongitude = max_lon[a], maxdepth = 30, minmagnitude = 2)))
    except:
        pass #no point pushing out another list of everything that has no quakes
twomag = [x for x in twoplus if len(x[1]) > 3]
if UTCDateTime(today).isoweekday() == 7: #if UTCDate = Sund, i.e NZ = Monday 
    for a in range(len(volcano_list)): # as above except weeks worth of data 
        try:
            weekly.append((volcano_list[a],
            client.get_events(starttime = UTCDateTime(today) - 604800,
            endtime = UTCDateTime(today), minlatitude = min_lat[a], 
            maxlatitude = max_lat[a], minlongitude = min_lon[a], 
            maxlongitude = max_lon[a], maxdepth = 30, minmagnitude = 2)))
        except:
            pass    
weeklymag = [x for x in weekly if len(x[1]) > 20] #if 20+ M> 2.
#Main outputs below - to keep them all popping out in the same place
print("\n") # new line for good formatting.
for x in range(len(anymag)):
    print("The following volcanoes had more than 10 EQs in past 24 h: " +
    anymag[x][0] + ". Count: " + str(len(anymag[x][1])))
for x in range(len(twomag)):
    print("The following volcanoes had more than 3 M2+ EQs in past 24 h: " +
    twomag[x][0] + ". Count: " +  str(len(twomag[x][1]))) #prints all M 2 +  
for x in range(len(weeklymag)):
    print("The following volcanoes had more than 20 M2+ EQs in past week: " + 
    weeklymag[x][0] + ". Count: " +str(len(weeklymag[x][1]))) #prints weekly m2 
print("\n")
# Calling functions to create graphs. The for loops allow multiple graphs/maps 
# to be created if multiple areas experience swarms on the same day.
for a in range(len(anymag)): # for loop allows multiple volcanoes
    magtime(anymag[a][1], anymag[a][0], yesterday)
    if anymag[a][0] == "White Island": #sp only works for White Island
        sp(anymag[a][1], yesterday)
    else:
        pass
    histmags(anymag[a][1], anymag[a][0])
    latlon(anymag[a][1], anymag[a][0], anymag[a])
# only displays these plots if above above doesn't run
if anymag == [] and twomag != []: 
    try:
        for a in range(len(twomag)):
            magtime(twomag[a][1], twomag[a][0], yesterday)
            histmags(twomag[a][1], twomag[a][0])
            latlon(twomag[a][1], twomag[a][0], twomag[a])
    except:
        pass
for a in range(len(weeklymag)):
    magtime(weeklymag[a][1], weeklymag[a][0], str(UTCDateTime(today) - 604800)[0:10])
    if weeklymag[a][0] == "White Island": #sp only works for White Island
        sp(weeklymag[a][1], str(UTCDateTime(today) - 604800)[0:10])
    else:
        pass
    histmags(weeklymag[a][1], weeklymag[a][0])
    latlon(weeklymag[a][1], weeklymag[a][0], weeklymag[a])